In [2]:
import json
import numpy as np

### 유사성 점수 계산 함수

In [1]:
# 사용자 1과 사용자 2 간의 피어슨 상관관계 점수 계산
def pearson_score(dataset, user1, user2):\
		# 마찬가지로 사용자가 없다면 오류 발생
    if user1 not in dataset:
        raise TypeError('Cannot find ' + user1 + ' in the dataset')

    if user2 not in dataset:
        raise TypeError('Cannot find ' + user2 + ' in the dataset')

    # 사용자 1과 사용자 2가 평가한 영화 추적 
    common_movies = {}

    for item in dataset[user1]:
        if item in dataset[user2]:
            common_movies[item] = 1

    num_ratings = len(common_movies) 

    # 공통으로 평가한 영화가 없다면 유사성 점수 = 0
    if num_ratings == 0:
        return 0

    # 두 사용자 모두가 평가한 영화의 평점의 합을 각각 계산
    user1_sum = np.sum([dataset[user1][item] for item in common_movies])
    user2_sum = np.sum([dataset[user2][item] for item in common_movies])

    # 마찬가지로 공통된 영화의 평점 제곱의 합을 각각 계산
    user1_squared_sum = np.sum([np.square(dataset[user1][item]) for item in common_movies])
    user2_squared_sum = np.sum([np.square(dataset[user2][item]) for item in common_movies])

    # 공통 영화에 대한 두 유저의 평점 곱의 합을 계산
    sum_of_products = np.sum([dataset[user1][item] * dataset[user2][item] for item in common_movies])

    # 위의 값들을 통해 피어슨 상관관계 점수 계산
    Sxy = sum_of_products - (user1_sum * user2_sum / num_ratings)
    Sxx = user1_squared_sum - np.square(user1_sum) / num_ratings
    Syy = user2_squared_sum - np.square(user2_sum) / num_ratings
    
		# 편차가 없다면 점수 = 0
    if Sxx * Syy == 0:
        return 0
		
		# 피어슨 점수 반환
    return Sxy / np.sqrt(Sxx * Syy)

### 협업 필터링 함수 

In [3]:
# 데이터 세트에서 input 사용자와 유사한 사용자 탐색하는 함수 
def find_similar_users(dataset, user, num_users):
    if user not in dataset:
        raise TypeError('Cannot find ' + user + ' in the dataset')

    # input 사용자와 데이터 세트 내의 모든 사용자 간의 피어슨 점수 계산 
    scores = np.array([[x, pearson_score(dataset, user, 
            x)] for x in dataset if x != user])

    # 내림차순으로 점수 정렬
    scores_sorted = np.argsort(scores[:, 1])[::-1]

    # 상위 'num_users' 개의 점수를 추출 후 return 
    top_users = scores_sorted[:num_users] 
    return scores[top_users] 

### 영화 추천 시스템

In [7]:
# input_user에 대한 영화 추천 진행 
def get_recommendations(dataset, input_user):
    if input_user not in dataset:
        raise TypeError('Cannot find ' + input_user + ' in the dataset')

    overall_scores = {}
    similarity_scores = {}
    
    # input_user 제외 모든 유저에 대해서 유사성 점수 계산 
    for user in [x for x in dataset if x != input_user]:
        similarity_score = pearson_score(dataset, input_user, user)
        
        # 유사성 점수가 0이라면 pass
        if similarity_score <= 0:
            continue
        
        # 현재 사용자가 평가했지만 입력 사용자는 평가하지 않은 영화 추출
        filtered_list = [x for x in dataset[user] if x not in \
                dataset[input_user] or dataset[input_user][x] == 0]

        # 필터링한 영화 목록 내 항목에 대해 유사성 점수를 기반으로 가중치가 부여된 평점, 유사성 점수 추적
        for item in filtered_list: 
            overall_scores.update({item: dataset[user][item] * similarity_score})
            similarity_scores.update({item: similarity_score})

    # 평점을 낼 내용이 없다면 추천 불가능 
    if len(overall_scores) == 0:
        return ['No recommendations possible']

    # 정규화에 의해 영화 순위 생성 
    movie_scores = np.array([[score/similarity_scores[item], item] 
            for item, score in overall_scores.items()])

    # 내림차순으로 점수 정렬 
    movie_scores = movie_scores[np.argsort(movie_scores[:, 0])[::-1]]

    # 영화 추천 추출
    movie_recommendations = [movie for _, movie in movie_scores]

    return movie_recommendations


In [10]:

user = "Chris Duncan"
ratings_file = 'ratings.json'

with open(ratings_file, 'r') as f:
    data = json.loads(f.read())

print("\nMovie recommendations for " + user + ":")
movies = get_recommendations(data, user) 
for i, movie in enumerate(movies):
    print(str(i+1) + '. ' + movie)


Movie recommendations for Chris Duncan:
1. Vertigo
2. Scarface
3. Goodfellas
4. Roman Holiday


In [11]:
user = "Julie Hammel"

print("\nMovie recommendations for " + user + ":")
movies = get_recommendations(data, user) 
for i, movie in enumerate(movies):
    print(str(i+1) + '. ' + movie)


Movie recommendations for Julie Hammel:
1. The Apartment
2. Vertigo
3. Raging Bull
